In [ ]:
%load_ext autoreload
%autoreload 2
import globvars as gv
from globvars import gp

In [ ]:
#$RS --mods msrc --feat_types H_act --parcel_group_names Sensorimotor,OccipitalI    nf,FrontalSup,FrontalInf,TemporalMid  --prefix onlyH_act_SMyOIyFSyFIyTM_noLFP

# Set params

In [ ]:
inc_S03 = True
inc_extra = True
inc_noMvt = inc_extra # inc_noMVt is a legacy name, left for compat
join_medcond = False  # medconds merged
join_subjects = False

rawnames_dict = {}

#fname_runstr = '_runstrings_ML_test.txt'
fname_runstr = '_runstrings_genfeats.txt'

comment = 'Per subject brain area significance, both MEG and LFP sampling 256Hz, no HFO'

runpars = {}
runpars['inc_S03'] = inc_S03
runpars['inc_extra'] = inc_extra
runpars['inc_noMvt'] = inc_extra
runpars['join_subjects'] =join_subjects
runpars['join_medcond'] = join_medcond
runpars['fname_runstr'] = fname_runstr
runpars['run_specific_args'] = '--load_TFR 0 --load_CSD 0 --load_TFRCSD_max_age_h 240'  #techincal params about inclusion of some algortihms or no
runpars['run_specific_args'] += ' --useHFO 0 --feat_types Hjorth'
# runpars['input_output_subdir_pairs'] =\
#     [('feats_wholectx_LFP256_covmat_entire',
#           'feats_wholectx_LFP256_covmat_entire'),
#      ('feats_wholectx_LFP256_covmat_rest',
#          'feats_wholectx_LFP256_covmat_rest')]

runpars['run_specific_args'] += ' --prescale_data 0'
runpars['run_specific_args'] += ' --Kalman_smooth 1'

# runpars['input_output_subdir_pairs'] =\
#     [('feats_wholectx_LFP256_covmat_entire_noprescale',
#           'feats_wholectx_LFP256_covmat_entire_noprescale'),
#      ('feats_wholectx_LFP256_covmat_rest_noprescale',
#          'feats_wholectx_LFP256_covmat_rest_noprescale')]

runpars['input_output_subdir_pairs'] =\
    [('feats_wholectx_LFP256_SSS_covmat_rest',
         'feats_wholectx_LFP256_SSS_covmat_rest')]

runpars['comment'] = comment

Q should I create entire new tree with same sampling rate? Or just top level? Or just bottm level?

params in ini file
    params in ini sub file
overriden params in runstring (inc folder name)
beh of py file

todo: add column in genfeats that codes subject id to separate them easier later
maybe in prep dat too
todo: maybe run genfeats for different grouping when computing stats and cov matrix

I run genfeats for each SESSION separately (so two raw files each time, one for each task)
for scaling two main params are important -- which interval I use, which is combine_type
baseline int used body_side_for_baseline_int, which is mostly = body_move_side.
Rescaling is done based on pooled data from single interval type collected from both raws within same session (fixed medcond fixed subject)

I could vary whether I use rest interval or entire

add force all artifacts option

should I really tune params when selecting best LFP? Perhaps not

TODO: make sure I sue same artifacts in best LFP sel and in ML later.
    So best LFP sel should care about MEG artif then
TODO: sync folds somehow

it would be nice to clearly see in run_ML whether 
rescaling was done in genfeats at what kind of combine was there
and also to be able to ge this info after running ML, on data collection stage
uses baseline_int from params


# Apply params

In [ ]:
rawnames_dict['off_Mvt_best'] = ['S01_off_hold',
 'S01_off_move',
 'S02_off_hold',
 'S02_off_move',
 'S04_off_hold',
 'S04_off_move',
 'S05_off_hold',
 'S05_off_move',
 'S07_off_hold',
 'S07_off_move']
rawnames_dict['off_unsure'] = ['S03_off_hold', 'S03_off_move']
rawnames_dict['off_Mvt'] = rawnames_dict['off_Mvt_best'] + rawnames_dict['off_unsure']
rawnames_dict['on_Mvt'] = ['S01_on_hold',
 'S01_on_move',
 'S02_on_hold',
 'S02_on_move',
 'S04_on_hold',
 'S04_on_move',
 'S05_on_hold',
 'S05_on_move',
 'S07_on_hold',
 'S07_on_move'] 

#1,2,3,4,5,7 -- good
#6 is bad
rawnames_dict['off_extra'] = ['S07_off_hold','S07_off_move','S09_off_rest','S10_off_rest', 'S10_off_move',
                              'S08_off_rest']
rawnames_dict['on_extra'] = ['S08_on_rest']

rns = rawnames_dict['off_Mvt_best'] + rawnames_dict['on_Mvt']
z = zip(rns[::2], rns[1::2])
per_subj_tasks_merged = list(z)
per_subj_tasks_merged = [ rawnames_dict['off_unsure'] ] + per_subj_tasks_merged
##############################################
rawnames_list = []
if join_subjects:
    if not join_medcond:
        if inc_S03:
            rawnames_list += [ ('normal', rawnames_dict['off_Mvt'] ) ] 
        else:
            rawnames_list += [ ('normal', rawnames_dict['off_Mvt_best'] ) ] 
        if inc_noMvt:
            raise ValueError('not implemented')

        rawnames_list += [ ('normal',rawnames_dict['on_Mvt']) ] 

        if inc_noMvt:
            raise ValueError('not implemented')
    else:
        if inc_S03:
            kk = 'off_Mvt'
        else:
            kk = 'off_Mvt_best'
        rns = rawnames_dict[kk] + rawnames_dict['on_Mvt']

        rawnames_list += [ ('normal',rns ) ] 
else:
    if not join_medcond:
        for tpl in per_subj_tasks_merged:
            rawnames_list += [ ('normal', list(tpl) ) ] 
    else:
        raise ValueError('Not implemented yet')

##############################################
if join_subjects:
    if join_medcond:
        rn_test =  ('test',[ 'S01_off_hold', 'S04_off_hold', 'S05_off_move'] ) 
    else:
        rn_test =  ('test',[ 'S01_off_hold', 'S01_on_move', 'S04_off_hold', 'S05_on_move'] ) 
else:
    if not join_medcond:
        rn_test =  ('test',[ 'S04_off_hold', 'S04_off_move'] ) 
    else:
        rn_test =  ('test',[ 'S04_off_hold', 'S04_on_move'] ) 
rawnames_list += [rn_test]
##############################################
pfstr_per_rnt = {}

if join_subjects:
    if join_medcond:
        pfile_str = ' --param_file ML_joint2_one_LFP_HPC.ini'
        #TODO: create a new pfile for testing, this one does not exit
        pfile_str_test = ' --param_file ML_joint2_one_LFP_HPC_fast.ini'
    else:
        pfile_str = ' --param_file ML_joint_one_LFP_HPC.ini'
        pfile_str_test = ' --param_file ML_joint_one_LFP_HPC_fast.ini' 
else:
    if not join_medcond:
        pfile_str      = ' --param_file genfeats_wholectx_HPC.ini'
        pfile_str_test = ' --param_file genfeats_wholectx_HPC_fast.ini' 
    
pfstr_per_rnt ['normal'] = pfile_str
pfstr_per_rnt ['test']   = pfile_str_test

#############################################
BANDS_BETA = 'beta'
BANDS_GAMMA = 'gamma'
BANDS_TREMOR = 'tremor'    
    
    #rslist_cur += ['--feat_types H_act --mods LFP      --prefix onlyH_act_onlyLFP         ']        
    #rslist_cur += ['--feat_types H_act --mods LFP      --prefix onlyH_act_onlyLFP         ']        

In [ ]:
rawnames_list

# Set timestamp

In [ ]:
import time
import datetime
print(f'Set new timestamp at {datetime.datetime.now()}')
run_corresp_id = int( time.time() * 10000 )


In [ ]:
#rawnames_list

# Generate file

In [ ]:
modes

In [ ]:
save = True
special_globinds = []  
# these are globinds that don't take into account test strings that will be added in the beginning

runstrings_per_rnt = {}
for rnt in pfstr_per_rnt:
    runstrings_per_rnt[rnt ] = []
    
n_testrunstr_to_include = 1
runfile = 'run_genfeats.py' 
no_LFP_unless_explicit = 1
    
global_ind = 0
runstrings = []

for inpdir,outdir in runpars['input_output_subdir_pairs']:
    iostr = ''
    if inpdir is not None:
        iostr += f' --input_subdir {inpdir}'
    if outdir is not None:
        iostr += f' --output_subdir {outdir}'
    # Cycle over rawnames
    for rnt,rawnames in rawnames_list:
        rawstr = ','.join(rawnames)    
        s = runfile
        s+= f' -r {rawstr}' 
        s+= pfstr_per_rnt[rnt]
        #s+= f' --runCID {run_corresp_id}'
        s+= ' ' + runpars['run_specific_args']
        s+= iostr
        descr = None
        if len(tpl) > 2:
            descr = tpl[2]

        runstrings_per_rnt[rnt ] += [s]; 
        if rnt != 'test':
            global_ind += 1            
        #runstrings += runstrings_per_rnt['normal' ]
        
runstrings = runstrings_per_rnt['normal' ]
    
if n_testrunstr_to_include > 0:
    runstrings = runstrings_per_rnt['test'][:n_testrunstr_to_include] + runstrings

srs = set(runstrings)
assert len(srs) == len(runstrings),  f'there are repeating runstrings {len(srs)} , {len(runstrings)}'

####################
        
import globvars as gv
from os.path import join as pjoin
import json

if save:
    fname_full_runstr = pjoin(gv.code_dir, 'run', fname_runstr)
    with open( fname_full_runstr, 'w' ) as f:
        for s in runstrings:
            f.write(s + '\n')
    fname_full_runstr_archive = pjoin(gv.code_dir, 'run', f'__{fname_runstr[:-4]}_{run_corresp_id}.txt')
    with open( fname_full_runstr, 'w' ) as f:
        for s in runstrings:
            f.write(s + '\n')
            
    print(fname_full_runstr)
    print(fname_full_runstr_archive)

In [ ]:
rslist_cur

In [ ]:
prefixes_types

## find inds to recalc

In [ ]:
#to_recalc = [('S07_on', 'OccipitalMid')]
#to_recalc += [('S01_off', 'SupraMarginal')]
to_recalc = [('S01_off', 'SupraMarginal')]
to_recalc += [('S01_off',   'onlyH_act_LFPand_only13') ]# 'onlyH_act_LFPand_only13')]
#'S01_off': {'onlyH_act_LFPand_only13'}
to_recalc = [('S02_on',   'onlyH_act_only15') ]

rsis = []
for rn,name in to_recalc:
    key_founds = []
    for key,item in pref2pgn.items():
        if item[1] == name:
            key_founds += [key]
    if not len(key_founds) and (name in pref2pgn):
        key_founds = [name]
        roi = pref2pgn[name][1]
    else:
        roi = name
    print('key_founds = ', key_founds)
        
    for rsi,rs in enumerate(runstrings):
        if rs.find('_test') >= 0 or rs.find('_fast') >= 0:
            continue
        items = rs.split()
        a = items[1::2]
        b = items[2::2]
        d = dict( zip(a,b) )
        
        c1 = d['-r'].find(rn) >= 0 
        c2 = d['--prefix'] in key_founds
        
        if c1 and c2:
            rsis += [rsi]
            print(f'{rsi:4} = {rn} : {roi:18} -> {d["--prefix"]}')
            #print(rsi)
            
rsis = list(sorted(set(rsis)))
print('\nto be given to sbatch: ', rsis)

In [ ]:
import gc; gc.collect()

In [ ]:
key_found

In [ ]:
pref2pgn

In [ ]:
pref2pgn

In [ ]:
runpars

In [ ]:
pref2pgn

In [ ]:
#prefixes_types, pfstr_per_rnt ['normal']

In [ ]:
runstrings

In [ ]:
pref2pgn

In [ ]:
special_globinds

In [ ]:
len(runstrings)

In [ ]:
#special_globinds_mod = special_globinds
special_globinds_mod = [gi + n_testrunstr_to_include for gi in special_globinds]
#[runstrings[gi][120:-93] for gi in special_globinds_mod]
runstrings_sub = [runstrings[gi] for gi in special_globinds_mod]
display( runstrings_sub )
#special_globinds_mod
print( ','.join(map(str,special_globinds_mod) ) )

In [ ]:
len(runstrings)

In [ ]:
sis = []
for rsi,rs in enumerate(runstrings):
    #if rs.find('--mods LFP') >= 0:
    if rs.find('15') >= 0:
        sis += [rsi]
        print(rs)
print( ','.join(map(str,sis) ) )

In [ ]:
corresp

In [ ]:
runstrings_per_rnt['test' ]

In [ ]:
runstrings_per_rnt

In [ ]:
gv.data_dir

In [ ]:
pwd

In [ ]:
pref2pgn

In [ ]:
runstrings[0]

In [ ]:

#open()

In [ ]:
s